# Getting Started with Aurora Python SDK

This notebook provides a comprehensive introduction to the SLAMTEC Aurora Python SDK. You'll learn how to:

1. Install and import the SDK
2. Connect to an Aurora device
3. Retrieve basic device information
4. Get pose data
5. Handle connections properly

## Prerequisites

- Aurora device connected to the network
- Python 3.6+ with the Aurora SDK installed
- Device IP address (or auto-discovery)

## Installation

If you haven't installed the Aurora SDK yet, follow the installation instructions in the main README.

## Step 1: Import the SDK

First, let's import the Aurora SDK and check that it's working properly.

**Important Note:** If you've modified the Aurora SDK source code or are getting unexpected errors like "too many values to unpack", you may need to restart the Jupyter kernel to reload the updated module. Use: `Kernel → Restart Kernel` from the menu.

In [1]:
# Import the Aurora SDK
# IMPORTANT: If you get "too many values to unpack" errors, restart the kernel first!

import importlib
import sys
import gc

# Force complete module cleanup
modules_to_remove = [name for name in list(sys.modules.keys()) if name.startswith('slamtec_aurora_sdk')]
for module_name in modules_to_remove:
    print(f"Removing cached module: {module_name}")
    del sys.modules[module_name]

# Force garbage collection
gc.collect()

# Now import fresh modules
from slamtec_aurora_sdk import AuroraSDK
from slamtec_aurora_sdk.exceptions import AuroraSDKError, ConnectionError, DataNotReadyError

# Import additional libraries for data handling
import numpy as np
import time
from datetime import datetime

print("✅ Aurora SDK imported successfully!")
print(f"SDK Version: {AuroraSDK.__module__}")

# Test the API to verify it returns the correct number of values
try:
    test_sdk = AuroraSDK()
    # This should fail with ConnectionError, not unpack error
    test_sdk.data_provider.get_current_pose()
except ConnectionError:
    print("✅ API returns correct number of values (2)")
except ValueError as e:
    if "too many values to unpack" in str(e):
        print("❌ KERNEL RESTART REQUIRED!")
        print("   Go to: Kernel → Restart Kernel, then re-run this cell")
    else:
        print(f"❌ Unexpected error: {e}")
except Exception as e:
    print(f"✅ API working (got expected error: {type(e).__name__})")

✅ Aurora SDK imported successfully!
SDK Version: slamtec_aurora_sdk.aurora_sdk
✅ API returns correct number of values (2)


## 🚨 Quick Fix for "too many values to unpack" Error

**If you're seeing the "too many values to unpack (expected 2)" error:**

1. **STOP** - Don't continue with the other cells
2. **Restart the Jupyter kernel**: `Kernel → Restart Kernel` from the menu
3. **Re-run the import cell above** (Cell #2)
4. **Then continue** with the rest of the notebook

This error happens because Jupyter cached an old version of the Aurora SDK. Restarting the kernel clears all cached modules and loads the updated version.

**Alternative quick test** - Run this cell to check if the API is working:

```python
# Quick API test
try:
    temp_sdk = AuroraSDK()
    temp_sdk.data_provider.get_current_pose()
except ConnectionError:
    print("✅ API fixed - returns 2 values as expected")
except ValueError as e:
    if "too many values to unpack" in str(e):
        print("❌ Still cached - RESTART KERNEL and re-run import cell")
```

In [2]:
# Create Aurora SDK instance (session is created automatically)
sdk = AuroraSDK()

print("✅ Aurora SDK instance created!")
print("\nAvailable components:")
print(f"- Controller: {type(sdk.controller).__name__}")
print(f"- DataProvider: {type(sdk.data_provider).__name__}")
print(f"- MapManager: {type(sdk.map_manager).__name__}")
print(f"- LIDAR2DMapBuilder: {type(sdk.lidar_2d_map_builder).__name__}")
print(f"- EnhancedImaging: {type(sdk.enhanced_imaging).__name__}")
print(f"- FloorDetector: {type(sdk.floor_detector).__name__}")

✅ Aurora SDK instance created!

Available components:
- Controller: Controller
- DataProvider: DataProvider
- MapManager: MapManager
- LIDAR2DMapBuilder: LIDAR2DMapBuilder
- EnhancedImaging: EnhancedImaging
- FloorDetector: FloorDetector


## Step 3: Device Discovery

Before connecting, let's discover Aurora devices on the network.

In [3]:
# Discover Aurora devices
print("Discovering Aurora devices...")
devices = sdk.discover_devices(timeout=10.0)

if devices:
    print(f"✅ Found {len(devices)} Aurora device(s):")
    for i, device in enumerate(devices):
        print(f"\nDevice {i}:")
        print(f"  Name: {device['device_name']}")
        print(f"  Connection options:")
        for j, option in enumerate(device['options']):
            print(f"    {j}: {option['protocol']}://{option['address']}:{option['port']}")
else:
    print("❌ No Aurora devices found on the network")
    print("Make sure your Aurora device is:")
    print("  - Powered on")
    print("  - Connected to the same network")
    print("  - In the correct operating mode")

Discovering Aurora devices...
✅ Found 1 Aurora device(s):

Device 0:
  Name: Aurora Device 0
  Connection options:
    0: tcp://[fe80::ad94:89de:cef2:dcb4]:7447
    1: tcp://192.168.1.212:7447


## Step 4: Connect to Device

Now let's connect to an Aurora device. You can either:
1. Use auto-discovery (connects to first found device)
2. Connect to a specific IP address

**Method 1: Auto-connect to discovered device**

In [4]:
# Method 1: Connect using discovered device info
if devices:
    try:
        print(f"Connecting to first discovered device: {devices[0]['device_name']}")
        sdk.connect(device_info=devices[0])
        print("✅ Connected successfully using device discovery!")
    except ConnectionError as e:
        print(f"❌ Connection failed: {e}")
else:
    print("No devices found for auto-connection")

Connecting to first discovered device: Aurora Device 0
✅ Connected successfully using device discovery!


**Method 2: Connect to specific IP address**

If you know your device's IP address, you can connect directly:

In [5]:
# Method 2: Connect using IP address
# Uncomment and modify the IP address below if needed
device_ip = "192.168.11.1"  # Replace with your device's IP

try:
    if not sdk.is_connected():
        print(f"Connecting to device at {device_ip}...")
        sdk.connect(connection_string=device_ip)
        print("✅ Connected successfully using IP address!")
    else:
        print("✅ Already connected to device")
except ConnectionError as e:
    print(f"❌ Connection failed: {e}")
    print("\nTroubleshooting tips:")
    print("  - Check that the IP address is correct")
    print("  - Ensure the device is powered on")
    print("  - Verify network connectivity")
    print("  - Make sure no other application is using the device")

✅ Already connected to device


## Step 5: Get Device Information

Once connected, let's retrieve basic device information.

In [5]:
if sdk.is_connected():
    try:
        # Get device information
        device_info = sdk.get_device_info()
        
        print("📱 Device Information:")
        print(f"  Device Name: {device_info.device_name}")
        print(f"  Model: {device_info.device_model_string}")
        print(f"  Firmware Version: {device_info.firmware_version}")
        print(f"  Hardware Version: {device_info.hardware_version}")
        print(f"  Serial Number: {device_info.serial_number}")
        
        # Get SDK version info
        version_info = sdk.get_version_info()
        print(f"\n🔧 SDK Information:")
        print(f"  Version: {version_info.get('version_string', 'Unknown')}")
        
    except Exception as e:
        print(f"❌ Failed to get device info: {e}")
else:
    print("❌ Not connected to device. Please run the connection steps above first.")

📱 Device Information:
  Device Name: Aurora
  Model: A1M1
  Firmware Version: 2.0.0-beta1
  Hardware Version: 0.0.0
  Serial Number: 156F806B5F93F596AD4775291B6583C7

🔧 SDK Information:
  Version: 2.0.0-beta1


## Step 6: Get Current Pose

One of the most common operations is getting the current device pose (position and orientation).

In [7]:
if sdk.is_connected():
    try:
        print("Getting current pose...")
        
        # Get pose in SE3 format (position + quaternion)
        position, rotation, timestamp = sdk.data_provider.get_current_pose(use_se3=True)
        
        print("\n📍 Current Pose (SE3 format):")
        print(f"  Position (x, y, z): ({position[0]:.3f}, {position[1]:.3f}, {position[2]:.3f}) meters")
        print(f"  Rotation (qx, qy, qz, qw): ({rotation[0]:.3f}, {rotation[1]:.3f}, {rotation[2]:.3f}, {rotation[3]:.3f})")
        print(f"  Timestamp: {timestamp} ns")
        
        # Convert timestamp to readable format
        timestamp_seconds = timestamp / 1e9

        
        # Also get pose in Euler format
        position_euler, rotation_euler, timestamp_euler = sdk.data_provider.get_current_pose(use_se3=False)
        print(f"\n📐 Current Pose (Euler format):")
        print(f"  Position (x, y, z): ({position_euler[0]:.3f}, {position_euler[1]:.3f}, {position_euler[2]:.3f}) meters")
        print(f"  Rotation (roll, pitch, yaw): ({rotation_euler[0]:.3f}, {rotation_euler[1]:.3f}, {rotation_euler[2]:.3f}) radians")
        print(f"  Rotation (degrees): ({np.degrees(rotation_euler[0]):.1f}°, {np.degrees(rotation_euler[1]):.1f}°, {np.degrees(rotation_euler[2]):.1f}°)")
        print(f"  Timestamp: {timestamp_euler} ns")
        
    except DataNotReadyError:
        print("⏳ Pose data not ready yet. The device may still be initializing.")
        print("   Try running this cell again in a few seconds.")
    except Exception as e:
        print(f"❌ Failed to get pose: {e}")
else:
    print("❌ Not connected to device. Please run the connection steps above first.")

Getting current pose...

📍 Current Pose (SE3 format):
  Position (x, y, z): (1.061, 9.900, 0.316) meters
  Rotation (qx, qy, qz, qw): (-0.023, 0.024, -0.197, 0.980)
  Timestamp: 3208140541779 ns

📐 Current Pose (Euler format):
  Position (x, y, z): (1.061, 9.900, 0.316) meters
  Rotation (roll, pitch, yaw): (-0.055, 0.037, -0.398) radians
  Rotation (degrees): (-3.2°, 2.1°, -22.8°)
  Timestamp: 3208140541779 ns


## Step 7: Real-time Pose Monitoring

Let's monitor the pose in real-time for a few seconds to see how it changes.

In [8]:
if sdk.is_connected():
    print("📊 Real-time pose monitoring (10 seconds)...")
    print("Move your Aurora device to see the pose changes!\n")
    
    start_time = time.time()
    pose_count = 0
    
    try:
        while time.time() - start_time < 10.0:  # Monitor for 10 seconds
            try:
                position, rotation, timestamp = sdk.data_provider.get_current_pose(use_se3=True)
                
                pose_count += 1
                elapsed = time.time() - start_time
                
                # Print pose every 2 seconds
                if pose_count % 20 == 0 or pose_count == 1:  # Assuming ~10Hz pose rate
                    timestamp_sec = timestamp / 1e9
                    print(f"[{elapsed:.1f}s] Position: ({position[0]:.3f}, {position[1]:.3f}, {position[2]:.3f}) | Time: {timestamp_sec:.3f}s")
                
            except DataNotReadyError:
                pass  # Skip if data not ready
            
            time.sleep(0.1)  # 10Hz monitoring
        
        print(f"\n✅ Monitoring complete! Captured {pose_count} pose samples.")
        print(f"   Average rate: {pose_count / 10.0:.1f} Hz")
        
    except KeyboardInterrupt:
        print("\n⏹️ Monitoring stopped by user.")
else:
    print("❌ Not connected to device. Please run the connection steps above first.")

📊 Real-time pose monitoring (10 seconds)...
Move your Aurora device to see the pose changes!

[0.0s] Position: (3.416, 8.588, 0.110) | Time: 3211.341s
[1.9s] Position: (4.458, 7.062, -0.035) | Time: 3213.274s
[3.9s] Position: (6.187, 6.496, -0.102) | Time: 3215.274s
[5.9s] Position: (7.570, 6.395, -0.209) | Time: 3217.207s
[7.9s] Position: (8.513, 5.616, -0.312) | Time: 3219.274s
[9.9s] Position: (8.111, 5.013, -0.339) | Time: 3221.341s

✅ Monitoring complete! Captured 100 pose samples.
   Average rate: 10.0 Hz


## Step 8: Proper Cleanup

It's important to properly disconnect and clean up resources when you're done.

In [9]:
# Proper cleanup
try:
    if sdk.is_connected():
        print("Disconnecting from device...")
        sdk.disconnect()
        print("✅ Disconnected successfully")
    
    print("Releasing SDK resources...")
    sdk.release()
    print("✅ SDK resources released")
    
except Exception as e:
    print(f"⚠️ Cleanup warning: {e}")

print("\n🎉 Getting Started tutorial completed!")

Disconnecting from device...
✅ Disconnected successfully
Releasing SDK resources...
✅ SDK resources released

🎉 Getting Started tutorial completed!


## Alternative: Using Context Manager (Recommended)

The Aurora SDK supports Python context managers for automatic cleanup. This is the recommended approach:

In [10]:
# Context manager approach (recommended)
print("🔄 Demonstrating context manager approach...")

# Replace with your device IP
device_ip = "192.168.11.1"

try:
    with AuroraSDK() as sdk:  # Automatic session creation and cleanup
        print("✅ SDK created with context manager")
        
        # Connect to device
        sdk.connect(connection_string=device_ip)
        print("✅ Connected to device")
        
        # Get a pose sample
        position, rotation, timestamp = sdk.data_provider.get_current_pose()
        print(f"📍 Sample pose: ({position[0]:.3f}, {position[1]:.3f}, {position[2]:.3f})")
        print(f"   Timestamp: {timestamp} ns ({timestamp / 1e9:.3f}s)")
        
        # No need to manually disconnect or release!
        # The context manager handles cleanup automatically
        
except ConnectionError as e:
    print(f"❌ Connection failed: {e}")
except Exception as e:
    print(f"❌ Error: {e}")

print("✅ Context manager automatically handled cleanup!")

🔄 Demonstrating context manager approach...
✅ SDK created with context manager
✅ Connected to device
📍 Sample pose: (0.000, 0.000, 0.000)
   Timestamp: 0 ns (0.000s)
✅ Context manager automatically handled cleanup!


## Summary

In this notebook, you learned:

1. ✅ **Import the Aurora SDK** and verify installation
2. ✅ **Create SDK instances** and understand the component architecture
3. ✅ **Discover Aurora devices** on the network
4. ✅ **Connect to devices** using auto-discovery or IP address
5. ✅ **Retrieve device information** including model and firmware
6. ✅ **Get pose data** in both SE3 and Euler formats
7. ✅ **Monitor real-time poses** to see device movement
8. ✅ **Proper cleanup** and resource management
9. ✅ **Context manager usage** for automatic cleanup

## Troubleshooting

### Common Issues

**"ValueError: too many values to unpack (expected 2)"**

This error typically occurs when the Jupyter kernel has cached an older version of the Aurora SDK module. To fix:

1. **Restart the kernel:** Go to `Kernel → Restart Kernel` in the Jupyter menu
2. **Re-run the import cell:** Execute the SDK import cell again after restarting
3. **Clear module cache:** The import cell above includes automatic module cache clearing

**"ModuleNotFoundError: No module named 'slamtec_aurora_sdk'"**

1. Check that the Aurora SDK is properly installed
2. Make sure you're running the notebook from the correct directory
3. Verify the installation path includes the SDK modules

**"ConnectionError: Not connected to any device"**

1. Verify device IP, network connectivity, and device status
2. Check that the Aurora device is powered on and in the correct mode
3. Ensure no other application is using the device

**"DataNotReadyError: Pose data not ready"**

1. This is normal during device initialization
2. Wait a few seconds and try again
3. The device may still be starting up or calibrating

### Getting Help

For more detailed troubleshooting:
- Check the [API Documentation](../docs/index.md) 
- See the main [README](../README.md) troubleshooting section
- Review other tutorial notebooks for advanced usage patterns

## Next Steps

Now that you understand the basics, you can explore more advanced features:

- **[Camera and Images](02_camera_and_images.ipynb)** - Working with camera preview and tracking frames
- **[LiDAR and Point Clouds](03_lidar_and_point_clouds.ipynb)** - Processing LiDAR scan data
- **[Mapping and Navigation](04_mapping_and_navigation.ipynb)** - VSLAM and 2D mapping
- **[Enhanced Imaging](05_enhanced_imaging.ipynb)** - Depth camera and semantic segmentation
- **[Advanced Usage](06_advanced_usage.ipynb)** - Multi-threading, error handling, and optimization